# Todo
-get the data into a vector <br />
-look into 

In [5]:
import tensorflow as tf
import pandas as pd

In [11]:
file_path = 'data/1114147.csv'
df_raw_data = pd.read_csv(file_path, encoding="ISO-8859-1", low_memory=False)

In [12]:
df_raw_data.keys()

Index(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE',
       'REPORTTPYE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY',
       'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC',
       'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF',
       'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed',
       'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYPressureTendency', 'HOURLYPressureChange',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp',
       'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp',
       'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp',
       'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays',
       'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth',
       'DAILYAverageStatio

In [13]:
df_raw_data.head()

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:03813,MACON MIDDLE GA REGIONAL AIRPORT GA US,104.5,32.6847,-83.6527,2007-11-01 00:53,FM-15,NaN,10.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:03813,MACON MIDDLE GA REGIONAL AIRPORT GA US,104.5,32.6847,-83.6527,2007-11-01 01:00,FM-12,CLR:00,9.94,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:03813,MACON MIDDLE GA REGIONAL AIRPORT GA US,104.5,32.6847,-83.6527,2007-11-01 01:53,FM-15,NaN,7.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:03813,MACON MIDDLE GA REGIONAL AIRPORT GA US,104.5,32.6847,-83.6527,2007-11-01 02:53,FM-15,NaN,7.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:03813,MACON MIDDLE GA REGIONAL AIRPORT GA US,104.5,32.6847,-83.6527,2007-11-01 03:53,FM-15,NaN,7.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_raw_data.keys()

Index(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE',
       'REPORTTPYE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY',
       'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC',
       'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF',
       'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed',
       'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYPressureTendency', 'HOURLYPressureChange',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp',
       'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp',
       'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp',
       'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays',
       'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth',
       'DAILYAverageStatio

In [48]:
df_num_subset = df_raw_data.iloc[:, 2:]

In [49]:
df_num_subset.DATE = pd.to_datetime(df_num_subset.DATE)

In [50]:
df_num_subset.DATE

0        2007-11-01 00:53:00
1        2007-11-01 01:00:00
2        2007-11-01 01:53:00
3        2007-11-01 02:53:00
4        2007-11-01 03:53:00
5        2007-11-01 04:00:00
6        2007-11-01 04:53:00
7        2007-11-01 05:53:00
8        2007-11-01 06:53:00
9        2007-11-01 07:00:00
10       2007-11-01 07:53:00
11       2007-11-01 08:53:00
12       2007-11-01 09:53:00
13       2007-11-01 10:00:00
14       2007-11-01 10:53:00
15       2007-11-01 11:53:00
16       2007-11-01 12:53:00
17       2007-11-01 13:00:00
18       2007-11-01 13:53:00
19       2007-11-01 14:53:00
20       2007-11-01 15:53:00
21       2007-11-01 16:53:00
22       2007-11-01 17:53:00
23       2007-11-01 18:53:00
24       2007-11-01 19:00:00
25       2007-11-01 19:53:00
26       2007-11-01 20:53:00
27       2007-11-01 21:53:00
28       2007-11-01 22:00:00
29       2007-11-01 22:53:00
                 ...        
144050   2017-10-29 23:53:00
144051   2017-10-29 23:59:00
144052   2017-10-30 00:53:00
144053   2017-

In [51]:
# Trying to put the years more middle of the road here
df_num_subset['year_norm'] = (df_num_subset.DATE.dt.year - 2000) / 20
df_num_subset['month_norm'] = df_num_subset.DATE.dt.month / 12
df_num_subset['day_norm'] = df_num_subset.DATE.dt.day / 31
df_num_subset['time_norm'] = ((df_num_subset.DATE.dt.hour * 60) + df_num_subset.DATE.dt.minute) / 1440
df_num_subset['time_norm']

0         0.036806
1         0.041667
2         0.078472
3         0.120139
4         0.161806
5         0.166667
6         0.203472
7         0.245139
8         0.286806
9         0.291667
10        0.328472
11        0.370139
12        0.411806
13        0.416667
14        0.453472
15        0.495139
16        0.536806
17        0.541667
18        0.578472
19        0.620139
20        0.661806
21        0.703472
22        0.745139
23        0.786806
24        0.791667
25        0.828472
26        0.870139
27        0.911806
28        0.916667
29        0.953472
            ...   
144050    0.995139
144051    0.999306
144052    0.036806
144053    0.041667
144054    0.078472
144055    0.120139
144056    0.161806
144057    0.166667
144058    0.203472
144059    0.245139
144060    0.286806
144061    0.291667
144062    0.328472
144063    0.370139
144064    0.411806
144065    0.416667
144066    0.453472
144067    0.495139
144068    0.536806
144069    0.541667
144070    0.578472
144071    0.

In [52]:
df_num_subset.LATITUDE = df_num_subset.LATITUDE / 90
df_num_subset.LATITUDE

0         0.363163
1         0.363163
2         0.363163
3         0.363163
4         0.363163
5         0.363163
6         0.363163
7         0.363163
8         0.363163
9         0.363163
10        0.363163
11        0.363163
12        0.363163
13        0.363163
14        0.363163
15        0.363163
16        0.363163
17        0.363163
18        0.363163
19        0.363163
20        0.363163
21        0.363163
22        0.363163
23        0.363163
24        0.363163
25        0.363163
26        0.363163
27        0.363163
28        0.363163
29        0.363163
            ...   
144050    0.363163
144051    0.363163
144052    0.363163
144053    0.363163
144054    0.363163
144055    0.363163
144056    0.363163
144057    0.363163
144058    0.363163
144059    0.363163
144060    0.363163
144061    0.363163
144062    0.363163
144063    0.363163
144064    0.363163
144065    0.363163
144066    0.363163
144067    0.363163
144068    0.363163
144069    0.363163
144070    0.363163
144071    0.

In [53]:
df_num_subset.LONGITUDE = df_num_subset.LONGITUDE / 180
df_num_subset.LONGITUDE

0        -0.464737
1        -0.464737
2        -0.464737
3        -0.464737
4        -0.464737
5        -0.464737
6        -0.464737
7        -0.464737
8        -0.464737
9        -0.464737
10       -0.464737
11       -0.464737
12       -0.464737
13       -0.464737
14       -0.464737
15       -0.464737
16       -0.464737
17       -0.464737
18       -0.464737
19       -0.464737
20       -0.464737
21       -0.464737
22       -0.464737
23       -0.464737
24       -0.464737
25       -0.464737
26       -0.464737
27       -0.464737
28       -0.464737
29       -0.464737
            ...   
144050   -0.464737
144051   -0.464737
144052   -0.464737
144053   -0.464737
144054   -0.464737
144055   -0.464737
144056   -0.464737
144057   -0.464737
144058   -0.464737
144059   -0.464737
144060   -0.464737
144061   -0.464737
144062   -0.464737
144063   -0.464737
144064   -0.464737
144065   -0.464737
144066   -0.464737
144067   -0.464737
144068   -0.464737
144069   -0.464737
144070   -0.464737
144071   -0.

In [41]:
df_num_subset.REPORTTPYE.unique()

array(['FM-15', 'FM-12', 'SOD', 'FM-16', 'SY-MT', 'AUTO'], dtype=object)

In [3]:
tensorboard --logdir=/tmp/tensorflow/mnist/logs

SyntaxError: invalid syntax (<ipython-input-3-d9b2cfe45809>, line 1)

In [42]:
df_num_subset.head()

,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,...,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD,month_norm,day_norm,year_norm,time_norm
0,104.5,32.6847,-83.6527,2007-11-01 00:53:00,FM-15,NaN,10.00,NaN,56,13.3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.916667,0.032258,0.35,0.036806
1,104.5,32.6847,-83.6527,2007-11-01 01:00:00,FM-12,CLR:00,9.94,NaN,56,13.3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.916667,0.032258,0.35,0.041667
2,104.5,32.6847,-83.6527,2007-11-01 01:53:00,FM-15,NaN,7.00,NaN,55,12.8,...,NaN,NaN,NaN,NaN,NaN,NaN,0.916667,0.032258,0.35,0.078472
3,104.5,32.6847,-83.6527,2007-11-01 02:53:00,FM-15,NaN,7.00,NaN,53,11.7,...,NaN,NaN,NaN,NaN,NaN,NaN,0.916667,0.032258,0.35,0.120139
4,104.5,32.6847,-83.6527,2007-11-01 03:53:00,FM-15,NaN,7.00,NaN,53,11.7,...,NaN,NaN,NaN,NaN,NaN,NaN,0.916667,0.032258,0.35,0.161806


In [54]:
df_num_subset.keys()

Index(['ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'REPORTTPYE',
       'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE',
       'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF',
       'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC',
       'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection',
       'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYPressureTendency', 'HOURLYPressureChange',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp',
       'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp',
       'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp',
       'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays',
       'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth',
       'DAILYAverageStationPressure', 'DAILYAverageSe

In [55]:
df_raw_data.keys()

Index(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE',
       'REPORTTPYE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY',
       'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC',
       'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF',
       'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed',
       'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYPressureTendency', 'HOURLYPressureChange',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp',
       'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp',
       'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp',
       'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays',
       'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth',
       'DAILYAverageStatio

In [76]:
df2 = df_raw_data.drop(['STATION', 'STATION_NAME', 'REPORTTPYE', 'HOURLYSKYCONDITIONS', 'HOURLYPRSENTWEATHERTYPE',
                        'HOURLYPressureTendency', 'HOURLYPressureChange',
                        'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp',
                        'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp',
                        'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays'], axis=1)
df2 = df2.iloc[:, :21]

In [77]:
df2.keys()

Index(['ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'HOURLYVISIBILITY',
       'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF',
       'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC',
       'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection',
       'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYSunrise', 'DAILYSunset'],
      dtype='object')

In [79]:
df2.head()

,ELEVATION,LATITUDE,LONGITUDE,DATE,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,...,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYSunrise,DAILYSunset
0,104.5,32.6847,-83.6527,2007-11-01 00:53,10.00,56,13.3,54.0,12.4,53,...,90.0,0,000,0.0,29.73,30.11,0.00,30.12,652,1744
1,104.5,32.6847,-83.6527,2007-11-01 01:00,9.94,56,13.3,54.0,12.4,53,...,90.0,0,000,0.0,29.73,30.11,NaN,NaN,652,1744
2,104.5,32.6847,-83.6527,2007-11-01 01:53,7.00,55,12.8,53.0,11.9,52,...,90.0,0,000,0.0,29.72,30.10,0.00,30.11,652,1744
3,104.5,32.6847,-83.6527,2007-11-01 02:53,7.00,53,11.7,52.0,11.1,51,...,93.0,3,290,0.0,29.71,30.10,0.00,30.10,652,1744
4,104.5,32.6847,-83.6527,2007-11-01 03:53,7.00,53,11.7,51.0,10.8,50,...,89.0,5,270,0.0,29.71,30.09,0.00,30.10,652,1744


In [78]:
df2.HOURLYWindGustSpeed = df2.HOURLYWindGustSpeed.fillna(0)

In [85]:
df2.DATE = pd.to_datetime(df2.DATE)

In [86]:
df2.DATE

0        2007-11-01 00:53:00
1        2007-11-01 01:00:00
2        2007-11-01 01:53:00
3        2007-11-01 02:53:00
4        2007-11-01 03:53:00
5        2007-11-01 04:00:00
6        2007-11-01 04:53:00
7        2007-11-01 05:53:00
8        2007-11-01 06:53:00
9        2007-11-01 07:00:00
10       2007-11-01 07:53:00
11       2007-11-01 08:53:00
12       2007-11-01 09:53:00
13       2007-11-01 10:00:00
14       2007-11-01 10:53:00
15       2007-11-01 11:53:00
16       2007-11-01 12:53:00
17       2007-11-01 13:00:00
18       2007-11-01 13:53:00
19       2007-11-01 14:53:00
20       2007-11-01 15:53:00
21       2007-11-01 16:53:00
22       2007-11-01 17:53:00
23       2007-11-01 18:53:00
24       2007-11-01 19:00:00
25       2007-11-01 19:53:00
26       2007-11-01 20:53:00
27       2007-11-01 21:53:00
28       2007-11-01 22:00:00
29       2007-11-01 22:53:00
                 ...        
144050   2017-10-29 23:53:00
144051   2017-10-29 23:59:00
144052   2017-10-30 00:53:00
144053   2017-

In [80]:
df3 = df2.dropna()

In [82]:
df3.size

1825026

In [83]:
df2.size

3025680

In [87]:
df3.keys()

Index(['ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'HOURLYVISIBILITY',
       'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF',
       'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC',
       'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection',
       'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYSunrise', 'DAILYSunset'],
      dtype='object')

In [88]:
df3.DATE

0         2007-11-01 00:53
2         2007-11-01 01:53
3         2007-11-01 02:53
4         2007-11-01 03:53
6         2007-11-01 04:53
7         2007-11-01 05:53
8         2007-11-01 06:53
10        2007-11-01 07:53
11        2007-11-01 08:53
12        2007-11-01 09:53
14        2007-11-01 10:53
15        2007-11-01 11:53
16        2007-11-01 12:53
18        2007-11-01 13:53
19        2007-11-01 14:53
20        2007-11-01 15:53
21        2007-11-01 16:53
22        2007-11-01 17:53
23        2007-11-01 18:53
25        2007-11-01 19:53
26        2007-11-01 20:53
27        2007-11-01 21:53
29        2007-11-01 22:53
30        2007-11-01 23:53
32        2007-11-02 00:53
34        2007-11-02 01:53
35        2007-11-02 02:53
36        2007-11-02 03:53
38        2007-11-02 04:53
39        2007-11-02 05:53
                ...       
144039    2017-10-29 15:53
144041    2017-10-29 16:53
144042    2017-10-29 17:53
144043    2017-10-29 18:53
144045    2017-10-29 19:53
144046    2017-10-29 20:53
1

In [89]:
df3.head()

,ELEVATION,LATITUDE,LONGITUDE,DATE,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,...,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYSunrise,DAILYSunset
0,104.5,32.6847,-83.6527,2007-11-01 00:53,10.00,56,13.3,54.0,12.4,53,...,90.0,0,000,0.0,29.73,30.11,0.00,30.12,652,1744
2,104.5,32.6847,-83.6527,2007-11-01 01:53,7.00,55,12.8,53.0,11.9,52,...,90.0,0,000,0.0,29.72,30.10,0.00,30.11,652,1744
3,104.5,32.6847,-83.6527,2007-11-01 02:53,7.00,53,11.7,52.0,11.1,51,...,93.0,3,290,0.0,29.71,30.10,0.00,30.10,652,1744
4,104.5,32.6847,-83.6527,2007-11-01 03:53,7.00,53,11.7,51.0,10.8,50,...,89.0,5,270,0.0,29.71,30.09,0.00,30.10,652,1744
6,104.5,32.6847,-83.6527,2007-11-01 04:53,6.00,53,11.7,52.0,11.1,51,...,93.0,3,270,0.0,29.71,30.09,0.00,30.10,652,1744


In [101]:
x = pd.to_datetime(df3.DATE)
x = x.dt.hour

In [102]:
x.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int64)

In [103]:
x.asfreq()

TypeError: asfreq() missing 1 required positional argument: 'freq'

In [104]:
x.value_counts()

16    3630
13    3630
14    3627
15    3627
11    3625
7     3624
10    3624
0     3623
19    3622
2     3622
1     3621
23    3621
12    3620
20    3620
6     3620
5     3619
4     3619
17    3618
22    3618
18    3617
3     3616
8     3615
21    3615
9     3613
Name: DATE, dtype: int64

In [105]:
df3.keys()

Index(['ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'HOURLYVISIBILITY',
       'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF',
       'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC',
       'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection',
       'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYSunrise', 'DAILYSunset'],
      dtype='object')

In [108]:
# Trying to put the years more middle of the road here
df3.DATE = pd.to_datetime(df3.DATE)
df3['year'] = df3.DATE.dt.year
df3['month'] = df3.DATE.dt.month
df3['day'] = df3.DATE.dt.day
df3['hour'] = df3.DATE.dt.hour

c:\users\dmunday6\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
c:\users\dmunday6\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\dmunday6\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWar

In [111]:
df3.keys()

Index(['ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'HOURLYVISIBILITY',
       'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF',
       'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC',
       'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection',
       'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYSunrise', 'DAILYSunset', 'year', 'month', 'day', 'hour'],
      dtype='object')

In [112]:
df3 = df3.iloc[:, 4:]

In [113]:
df3.keys()

Index(['HOURLYVISIBILITY', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC',
       'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF',
       'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed',
       'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYSunrise', 'DAILYSunset', 'year', 'month', 'day', 'hour'],
      dtype='object')

In [114]:
df_go_time = df3[[
    'year', 
    'month',
    'day',
    'hour',
    'HOURLYWindDirection',
    'HOURLYWindGustSpeed',
    'HOURLYVISIBILITY',
    'HOURLYDRYBULBTEMPF',
    'HOURLYWETBULBTEMPF',
    'HOURLYDewPointTempF',
    'HOURLYRelativeHumidity',
    'HOURLYAltimeterSetting',
    'HOURLYSeaLevelPressure']]

In [115]:
df_go_time.keys()

Index(['year', 'month', 'day', 'hour', 'HOURLYWindDirection',
       'HOURLYWindGustSpeed', 'HOURLYVISIBILITY', 'HOURLYDRYBULBTEMPF',
       'HOURLYWETBULBTEMPF', 'HOURLYDewPointTempF', 'HOURLYRelativeHumidity',
       'HOURLYAltimeterSetting', 'HOURLYSeaLevelPressure'],
      dtype='object')

In [116]:
df_go_time.size

1129778